### 크롤링으로  업종별 코드 가져오기

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
import time
import urllib.request
from selenium.webdriver import Chrome
import json
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
from selenium import webdriver


# selenium을 위한 chrome 브라우저 켜기
browser  = Chrome()
browser.maximize_window()

annual_date = ['2015/12', '2016/12', '2017/12', '2018/12', '2019/12', '2020/12','2021/12', '2022/12']
# 각각 지표들 데이터프레임에 넣기 위한 초기 선언, (df_NI(Net Income) : 당기순이익을 뜻함)

url_up = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'

## 밑에는 업종별 코드를 반환하기 위한 코드 
res = requests.get(url_up)
html = res.text
soup = BeautifulSoup(html,'html.parser')

upjong = {}
rows = soup.select('td > a')

for r in range(len(rows)):
    upjong[soup.select('td > a')[r]['href'][43:]] = soup.select('td > a')[r].text
upjong_code = list(upjong.keys())[:-1]
print(upjong_code)

['234', '138', '173', '230', '42', '44', '227', '136', '196', '212', '36', '210', '134', '229', '144', '171', '232', '177', '191', '192', '12', '176', '20', '146', '154', '161', '168', '198', '190', '207', '226', '174', '215', '62', '185', '189', '199', '217', '109', '38', '187', '164', '203', '205', '216', '25', '204', '208', '162', '235', '197', '53', '35', '140', '219', '206', '220', '200', '167', '33', '145', '231', '228', '193', '221', '183', '214', '182', '202', '213', '211', '222', '233', '184', '236', '194', '166', '218', '201']


In [3]:
# 업종코드를 입력하면 --> 해당 기업 코드가 나오는 함수. 
def upjong_url(code):
    tickers = {}
    url_tmp = "https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=%s"
    url = url_tmp % (code)
    res = requests.get(url)
    html = res.text
    
    soup = BeautifulSoup(html,"html.parser")
    rows = soup.select('td > div')

    for r in rows[1:]:
        tickers[r.select('a')[0]['href'][20:]] = r.text
    a = [k for k in tickers.keys() if tickers[k][-2]=='우' or tickers[k][-3]=='우']
    # 우량주 종목 제외    
    for i in range(len(a)):
        del tickers[a[i]]

    code = list(tickers.keys())
    com = list(tickers.values())
    return code,com
#회사 코드를 사용해서 url완성하는 함수
def ttt(xx):
    name = xx
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    return base_url
# 종목코드를 통해 재무제표 크롤링
def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    browser.get(base_url)
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    
    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    #print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    return pd.DataFrame(td2,columns = col,index = date).T
# numpy -> pandas 데이터프레임으로 변환하는 사용자 정의 함수
def trans_df(ar,com):
    #df = np.array([])
     
    ar.resize((int(len(ar)/8),8))
    df = pd.DataFrame(data = ar,index=com,columns = annual_date)
    return df

In [4]:
# res가 업종별 5순위 종목을 누적해서 데이터프레임에 저장 업종코드 res라는 전혀 다른 의미.

res = pd.DataFrame([])
toplist = []
for up in upjong_code:
    # 업종별로 회사 코드를 받음, 전자장비및기기업종 -> 78개 회사 리스트를 받는다.
    print(up)
    df_PER,df_PBR,df_ROE,df_ROA,df_EPS,df_NI = np.array([]),np.array([]),np.array([]),np.array([]),np.array([]),np.array([])
    code, com = upjong_url(up)
    for c in code:
        # 재무제표를 종목별로 불러와 지표 저장
        df = stock_crawler(c)
        df_PER = np.hstack([df_PER,df.loc['PER(배)'].values])
        df_PBR = np.hstack([df_PBR,df.loc['PBR(배)'].values])
        df_EPS = np.hstack([df_EPS,df.loc['EPS(원)'].values])
        df_ROE = np.hstack([df_ROE,df.loc['ROE(%)'].values])
        df_ROA = np.hstack([df_ROA,df.loc['ROA(%)'].values])
        df_NI = np.hstack([df_NI,df.loc['당기순이익'].values])
    # 데이터프레임으로 변환
    df_PER = trans_df(df_PER,com)
    df_PBR = trans_df(df_PBR,com)
    df_EPS = trans_df(df_EPS,com)
    df_ROE = trans_df(df_ROE,com)
    df_ROA = trans_df(df_ROA,com)
    df_NI = trans_df(df_NI,com)
    # 지표별로 랭크 구하여 합산하기
    df_rank = df_PER.rank(method='min')+df_PBR.rank(method='min')+df_EPS.rank(method='max')+df_ROE.rank(method='max')+df_ROA.rank(method='max')
    +df_NI.rank(method='max')
    # 연도
    sc ='2019/12'
    df_rank.sort_values(by=sc,inplace=True)
    res = pd.concat([res,df_rank.head()])
    # 업종별로 랭크가 작은 상위 5위권 항목들 리스트 보완해야 함.
    #toplist.append(list(df_rank.head().index))

234


C:\Users\No\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: use driver.switch_to.frame instead


IndexError: list index out of range

In [ ]:
toplist = list(np.array(toplist).flatten())